In [9]:
from shapely.geometry import Point

from shapely.geometry import Polygon

from shapely.geometry import MultiPolygon

from shapely.geometry import mapping

import psycopg2
import requests 
from shapely.geometry import Point,Polygon,MultiPolygon,mapping
import datetime
import pandas as pd

from shapely.wkb import loads
from shapely.wkt import dumps, loads
import datetime
import json
from ipyleaflet import (
    Map, Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON
)
import random
import datetime
from datetime import datetime

OSError: [WinError 126] Не найден указанный модуль

In [55]:
# DB connection

connection = psycopg2.connect(dbname = 'geo',
                              user = 'postgres',
                              password = 'a')

cursor = connection.cursor()

In [ ]:
# Create tables in Postgres

cursor.execute("CREATE EXTENSION postgis")

cursor.execute("CREATE TABLE areacommand (id SERIAL PRIMARY KEY, name VARCHAR(20), geom GEOMETRY)")

cursor.execute("CREATE TABLE beats (id SERIAL PRIMARY KEY, beat VARCHAR(6), agency VARCHAR(3), areacomm VARCHAR(15),geom GEOMETRY)")

cursor.execute("CREATE TABLE incidents (id SERIAL PRIMARY KEY, address VARCHAR(72), crimetype VARCHAR(255), date DATE,geom GEOMETRY)")

connection.commit()

In [ ]:
# Data for first table

url = 'http://coagisweb.cabq.gov/arcgis/rest/services/public/adminboundaries/MapServer/8/query'

params = {"where":"1=1",
          "outFields":"*",
          "outSR":"4326",
          "f":"json"}

r = requests.get(url,params=params)
data = r.json()

# Polygon Creation

for acmd in data['features']:
    polys = []
    
    for ring in acmd['geometry']['rings']:
        polys.append(Polygon(ring))
    p = MultiPolygon(polys)
    name = acmd['attributes']['Area_Command']
    
cursor.execute("INSERT INTO areacommand (name, geom) VALUES ('{}', ST_GeomFromText('{}'))".format(name, p.wkt))

connection.commit()

In [ ]:
# Data for the second Table

url='http://coagisweb.cabq.gov/arcgis/rest/services/public/adminboundaries/MapServer/9/query'
params={"where":"1=1","outFields":"*","outSR":"4326","f":"json"}
r=requests.get(url,params=params)
data=r.json()

for acmd in data['features']:
    polys=[]
    for ring in acmd['geometry']['rings']:
        polys.append(Polygon(ring))
    p=MultiPolygon(polys)
 
    beat = acmd['attributes']['BEAT']
    agency = acmd['attributes']['AGENCY']
    areacomm = acmd['attributes']['AREA_COMMA']

    cursor.execute("INSERT INTO beats (beat, agency,areacomm,geom) VALUES ('{}','{}','{}', ST_GeomFromText('{}'))".format(beat,agency,areacomm,p.wkt))

connection.commit()

In [ ]:
# Data for the third Table

url='http://coagisweb.cabq.gov/arcgis/rest/services/public/APD_Incidents/MapServer/0/query'
params={"where":"1=1","outFields":"*","outSR":"4326","f":"json"}
r=requests.get(url,params=params)
data=r.json()

for a in data["features"]:
    address=a["attributes"]["BlockAddress"]
    crimetype=a["attributes"]["IncidentType"]
    if a['attributes']['date'] is None:
        pass
    else:
        date = datetime.datetime.fromtimestamp(a['attributes']['date'] / 1e3).date()
    try:
        p=Point(float(a["geometry"]["x"]),float(a["geometry"]["y"]))
        cursor.execute("INSERT INTO incidents (address,crimetype,date, geom) VALUES ('{}','{}','{}', ST_GeomFromText('{}'))".format(address,crimetype,str(date), p.wkt))
    except KeyError:
        pass
    
connection.commit()

In [63]:
# Map

center = [40.8540630260003,-73.9047838229999]
zoom = 10
map = Map(center=center, zoom=zoom)
map

Map(center=[40.8540630260003, -73.9047838229999], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

In [14]:
# Query with condition

cursor = connection.cursor()

cursor.execute(""" SELECT address, crimetype, date, ST_AsGeoJSON(geom)
                   FROM incidents
                   WHERE date > '2018-12-29'
                   LIMIT 100""" )
               
result=cursor.fetchall()

for i in result:
    layer=json.loads(i[3])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [13]:
# Add Polygons from commands
cursor = connection.cursor()

cursor.execute("SELECT name, ST_AsGeoJSON(geom) from areacommand")

c=cursor.fetchall()

for x in c:
    layer=json.loads(x[1])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)

cursor.close()

In [ ]:
# Add areas

cursor.execute("SELECT beat, ST_AsGeoJSON(geom) from beats")

c=cursor.fetchall()

for x in c:
    layer=json.loads(x[1])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [10]:
# Points inside a Polygon

cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(i.geom)
                   FROM incidents i JOIN areacommand acmd ON ST_Intersects(acmd.geom, i.geom) 
                   WHERE acmd.name = 'VALLEY'
                   LIMIT 10 """ )

crimes = cursor.fetchall()

for j in crimes:
    layer=json.loads(j[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [15]:
# Inside Different Polygons

cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(geom)
                   FROM beats where beats.beat in ('336','523','117','226','638','636')""")

area=cursor.fetchall()

for i in area:
    layer=json.loads(i[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)

cursor.execute(""" SELECT ST_AsGeoJSON(i.geom) 
                   FROM incidents i JOIN beats b ON ST_Intersects(b.geom, i.geom) 
                   WHERE b.beat in ('336','523','117','226','638','636')
                   LIMIT 30 """)

crime=cursor.fetchall()

for i in crime:
    layer=json.loads(i[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)

cursor.close()

In [16]:
# Inside a certain radius


p = Point([-106.578677,35.062485])
pgeojson=mapping(p)
player=GeoJSON(data=pgeojson)
map.add_layer(player)

cursor = connection.cursor()

cursor.execute("SELECT ST_AsGeoJSON(ST_Buffer(ST_GeomFromText('{}')::geography,1500));".format(p.wkt))

radius=cursor.fetchall()

buffer=json.loads(radius[0][0])
bufferlayer=GeoJSON(data=buffer)
map.add_layer(bufferlayer)

In [17]:
cursor.execute("SELECT ST_AsText(ST_Buffer(ST_GeomFromText('{}')::geography,1500));".format(p.wkt))
bufferwkt=cursor.fetchall()
b=loads(bufferwkt[0][0])

In [18]:
cursor.execute(""" SELECT ST_AsGeoJSON(incidents.geom) 
                   FROM incidents 
                   WHERE ST_Intersects(ST_GeomFromText('{}'), incidents.geom)
                   LIMIT 15 """.format(b.wkt))

crime=cursor.fetchall()

for x in crime:
    layer=json.loads(x[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [ ]:
# Nearest neighbour

cursor = connection.cursor()

p = Point([35.145477, -106.651901])

cursor.execute(""" SELECT ST_AsGeoJSON(incidents.geom), ST_Distance(incidents.geom::geography,ST_GeometryFromText('{}')::geography) 
                   FROM incidents 
                   ORDER BY incidents.geom<->ST_GeometryFromText('{}')
                   LIMIT 15""".format(p.wkt,p.wkt))
c=cursor.fetchall()

for x in c:
    layer=json.loads(x[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)

cursor.close()

In [3]:
new_data = pd.read_csv('C:/Users/vladislav/Desktop/GIS/data/ny_crimes.csv')
new_data.head()

,date,time,region,age_group,vic_sex,cr_type,latitude,longitude
0,12/31/2019,23:15:00,MANHATTAN,25-44,M,gambling,40.800244,-73.953390
1,12/31/2019,20:14:00,BROOKLYN,25-44,M,fraud,40.660527,-73.917156
2,12/30/2019,21:29:00,BROOKLYN,25-44,M,theft,40.656923,-73.939647
3,12/30/2019,03:17:00,BROOKLYN,18-24,M,kidnapping,40.686815,-73.919370
4,12/30/2019,03:17:00,BROOKLYN,45-64,M,rape,40.686815,-73.919370


In [4]:
# Inserting other data to Postgres Tables

cursor = connection.cursor()

for i in range(0,len(new_data)):
    
    region = new_data['region'][i]
    time = pd.to_datetime(new_data['time'][i],format='%H:%M:%S').time()
    date = datetime.strptime(new_data['date'][i],'%m/%d/%Y').date()
    age = new_data['age_group'][i]
    sex = new_data['vic_sex'][i]
    type_cr = new_data['cr_type'][i]
    
    # For Leaflet map change the places latLon
    point = Point(float(new_data['longitude'][i]),float(new_data['latitude'][i]))
    
    cursor.execute( "insert into ny_crimes (region,age_group,vic_sex,cr_type,geom,date,time) values ('{}','{}','{}','{}','{}','{}','{}')".format( str(region), str(age), str(sex),
                                                                                                                                                  str(type_cr), point.wkt, str(date), str(time) ) )
connection.commit() 
cursor.close()

In [25]:
# Region (Polygons) for New York
cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(geom) 
                   FROM ny_parts
                   WHERE ny_parts.boro_name = 'Bronx' """ )

points = cursor.fetchall()

for i in points:
    layer = json.loads(i[0])
    layergeo = GeoJSON(data = layer)
    map.add_layer(layergeo)
        
cursor.close()

In [51]:
# Query for New York 
cursor = connection.cursor()

cursor.execute("""SELECT ST_AsGeoJSON(geom) 
                  FROM ny_crimes
                  WHERE region = 'STATEN ISLAND'
                  LIMIT 15 """ )

points = cursor.fetchall()

for i in points:
    layer = json.loads(i[0])
    layergeo = GeoJSON(data = layer)
    map.add_layer(layergeo)
        
cursor.close()

In [50]:
# Points inside a Polygon

cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(ny_crimes.geom)
                   FROM ny_crimes JOIN ny_parts ON ST_Intersects(ny_crimes.geom, ny_parts.geom) 
                   WHERE boro_name = 'STATEN ISLAND'
                   LIMIT 10 """ )

crimes = cursor.fetchall()

for j in crimes:
    layer=json.loads(j[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [36]:
# Inside Different Polygons

cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(ny_parts.geom)
                   FROM ny_parts
                   WHERE ny_parts.boro_name in ('Manhattan','Brooklyn') """)

areas = cursor.fetchall()

for i in areas:
    layer=json.loads(i[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    

cursor.execute(""" SELECT ST_AsGeoJSON(ny_crimes.geom)
                   FROM ny_crimes JOIN ny_parts ON ST_Intersects(ny_crimes.geom, ny_parts.geom) 
                   WHERE ny_parts.boro_name in ('Manhattan','Brooklyn')
                   LIMIT 30 """)

crime=cursor.fetchall()

for i in crime:
    layer=json.loads(i[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)

cursor.close()

In [64]:
# Inside a certain radius

p = Point([-74.162627,40.576205])
pgeojson=mapping(p)
player=GeoJSON(data=pgeojson)
map.add_layer(player)

cursor = connection.cursor()

cursor.execute("SELECT ST_AsGeoJSON(ST_Buffer(ST_GeomFromText('{}')::geography,5000));".format(p.wkt))

radius=cursor.fetchall()

buffer=json.loads(radius[0][0])
bufferlayer=GeoJSON(data=buffer)
map.add_layer(bufferlayer)

In [65]:
cursor = connection.cursor()

cursor.execute(""" SELECT ST_AsGeoJSON(ny_crimes.geom) 
                   FROM ny_crimes 
                   WHERE ST_Intersects(ST_GeomFromText('{}'), ny_crimes.geom)
                   LIMIT 15 """.format(b.wkt))

crime=cursor.fetchall()

for x in crime:
    layer=json.loads(x[0])
    layergeojson=GeoJSON(data=layer)
    map.add_layer(layergeojson)
    
cursor.close()

In [66]:
connection.close()